In [1]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import TensorDataset, DataLoader

In [3]:
column_names = ['ID', 'Game', 'Sentiment', 'Text']



In [4]:
train_df = pd.read_csv("twitter_training.csv",names=column_names)
val_df = pd.read_csv("twitter_validation.csv",names=column_names)

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         74682 non-null  int64 
 1   Game       74682 non-null  object
 2   Sentiment  74682 non-null  object
 3   Text       73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [6]:
train_df.head()

,ID,Game,Sentiment,Text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [7]:
train_df.drop_duplicates(subset=['Text'], inplace=True)
val_df.drop_duplicates(subset=['Text'], inplace=True)

In [8]:
train_df.dropna(subset=['Text'], inplace=True)
val_df.dropna(subset=['Text'], inplace=True)

In [9]:
label_encoder = LabelEncoder()
train_df['Sentiment'] = label_encoder.fit_transform(train_df['Sentiment'])
val_df['Sentiment'] = label_encoder.transform(val_df['Sentiment'])


In [10]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_df['Text']).toarray()
y_train = train_df['Sentiment'].values

In [11]:
X_val = vectorizer.transform(val_df['Text']).toarray()
y_val = val_df['Sentiment'].values

In [12]:
X_val.shape[1]

31062

In [13]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

# Convert NumPy arrays to PyTorch tensors for validation data
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create TensorDataset for training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

# Create TensorDataset for validation data
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# Define batch size
batch_size = 32

# Create DataLoader for training data
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create DataLoader for validation data
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # Input gate weights
        self.W_ix = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.W_ih = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_i = nn.Parameter(torch.Tensor(hidden_size))
        
        # Forget gate weights
        self.W_fx = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.W_fh = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_f = nn.Parameter(torch.Tensor(hidden_size))
        
        # Cell gate weights
        self.W_cx = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.W_ch = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_c = nn.Parameter(torch.Tensor(hidden_size))
        
        # Output gate weights
        self.W_ox = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.W_oh = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_o = nn.Parameter(torch.Tensor(hidden_size))
        
        self.reset_parameters()
        
    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.W_ix, a=0, mode='fan_in', nonlinearity='sigmoid')
        nn.init.kaiming_uniform_(self.W_ih, a=0, mode='fan_in', nonlinearity='sigmoid')
        nn.init.constant_(self.b_i, 0)
        
        nn.init.kaiming_uniform_(self.W_fx, a=0, mode='fan_in', nonlinearity='sigmoid')
        nn.init.kaiming_uniform_(self.W_fh, a=0, mode='fan_in', nonlinearity='sigmoid')
        nn.init.constant_(self.b_f, 0)
        
        nn.init.kaiming_uniform_(self.W_cx, a=0, mode='fan_in', nonlinearity='tanh')
        nn.init.kaiming_uniform_(self.W_ch, a=0, mode='fan_in', nonlinearity='tanh')
        nn.init.constant_(self.b_c, 0)
        
        nn.init.kaiming_uniform_(self.W_ox, a=0, mode='fan_in', nonlinearity='sigmoid')
        nn.init.kaiming_uniform_(self.W_oh, a=0, mode='fan_in', nonlinearity='sigmoid')
        nn.init.constant_(self.b_o, 0)
        
    def forward(self, x, prev_hidden):
        h_prev, c_prev = prev_hidden
        
        # Input gate
        i = torch.sigmoid(torch.matmul(x, self.W_ix) + torch.matmul(h_prev, self.W_ih) + self.b_i)
        
        # Forget gate
        f = torch.sigmoid(torch.matmul(x, self.W_fx) + torch.matmul(h_prev, self.W_fh) + self.b_f)
        
        # Update cell state
        c_tilde = torch.tanh(torch.matmul(x, self.W_cx) + torch.matmul(h_prev, self.W_ch) + self.b_c)
        c = f * c_prev + i * c_tilde
        
        # Output gate
        o = torch.sigmoid(torch.matmul(x, self.W_ox) + torch.matmul(h_prev, self.W_oh) + self.b_o)
        
        # Update hidden state
        h = o * torch.tanh(c)
        
        return h, c

In [16]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.lstm_cell = LSTMCell(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        h = torch.zeros(batch_size, self.hidden_size, device=x.device)
        c = torch.zeros(batch_size, self.hidden_size, device=x.device)
        
        for i in range(seq_len):
            h, c = self.lstm_cell(x[:, i, :], (h, c))
        
        out = self.fc(h)
        return out


In [17]:
input_size = X_train.shape[1]  # Input size is the number of features
hidden_size = 128  # Number of units in the RNN layer
output_size = len(label_encoder.classes_)  # Number of classes (sentiments)



In [18]:
model = LSTM(input_size, hidden_size, output_size).to(device)

In [19]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [20]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs, device):
    best_accuracy = 0.0
    best_epoch = 0
    best_model_state = None
    
    for epoch in range(num_epochs):
        # Training loop
        model.train()
        total_loss = 0
        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as t:
            for inputs, labels in t:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs.unsqueeze(1))  # Add an extra dimension for RNN input
                loss = criterion(outputs.squeeze(), labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                t.set_postfix(loss=total_loss / len(train_loader))        

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        total_correct = 0
        total_samples = 0

        with torch.no_grad():
            with tqdm(val_loader, desc="Validation", unit="batch") as t:
                for inputs, labels in t:
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    outputs = model(inputs.unsqueeze(1))  # Add an extra dimension for RNN input
                    _, predicted = torch.max(outputs, 1)
                    total_correct += (predicted == labels).sum().item()
                    total_samples += labels.size(0)
                    accuracy = total_correct / total_samples
                    t.set_postfix(accuracy=accuracy)

        # Check if the current model has the highest validation accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_epoch = epoch + 1
            best_model_state = model.state_dict().copy()
    
    # Load the best model parameters
    if best_model_state:
        model.load_state_dict(best_model_state)
        print(f"Best model details:\nEpoch: {best_epoch}\nValidation Accuracy: {best_accuracy}")
    
    return model


In [ ]:
num_epochs = 5
final_model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs, device)

Epoch 5/5:  63%|████████████████████████████████████████████████▍                            | 1366/2172 [05:06<02:55,  4.60batch/s, loss=0.0239]